In [3]:
import schach
import schach_darstellung as D
from ipycanvas import MultiCanvas
from time import sleep


layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas

D.draw_chessboard(bg)
pieces = schach.get_pieces()
D.place_pieces(fg, pieces)

schach.update = lambda event, **kwargs: D.update(fg, event, **kwargs)
schach.new_game()
mcanvas

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [2]:
moves = [('e2', 'e4'), ('e7', 'e5'), ('f1', 'c4'), ('f8', 'c5'),
         ('d1', 'h5'), ('g8', 'f6'), ('h5', 'f7')]
(' ', 4, 6), ('B', 4, 4)
schach.new_game()
for move in moves:
    schach.hmove(*move)
    sleep(1)

In [10]:
piece_sybmol = {
    'K': '♔',
    'D': '♕',
    'T': '♖',
    'L': '♗',
    'S': '♘',
    'B': '♙',
    'k': '♚',
    'd': '♛',
    't': '♜',
    'l': '♝',
    's': '♞',
    'b': '♟',
}
